In [ ]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)
import cwa

with open('env.json', encoding='utf-8') as f:
    env = json.load(f)

app = Flask(__name__)

configuration = Configuration(access_token=env.get('CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(env.get('CHANNEL_SECRET'))


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    print(event.message.text)  #
    print(event.source.user_id) #
    print(event.timestamp)
    print(event.reply_token)

    # ask = event.message.text
    # if ask == 'hello':
    #     ans = '我很好'
    # elif ask == 'hi':
    #     ans = '您哪位 ?'
    # else:
    #     ans = '我不知道妳再說甚麼'

    ask = event.message.text
    ask_map = {'hello': '我很好', 'hi': '您哪位'}
    #ans = ask_map.get(ask, '我不知道妳再說甚麼')
    ans = ask_map.get(ask)
    if not ans:
        ans = cwa.cwa2(ask, env.get('CWA_KEY'))
        ans = cwa.tostr(ans, '\n') or '無此站'
    
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=ans)]
            )
        )

if __name__ == "__main__":
    app.run(port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit


臺南
U091e03d020e37cce02ba9fd2f03377ec
1762584101185
0353f84d673542babae5b66b7eddff31


127.0.0.1 - - [08/Nov/2025 14:41:42] "POST /callback HTTP/1.1" 200 -


In [8]:
import time

t = 1762579212668
time.localtime(t/1000)

time.struct_time(tm_year=2025, tm_mon=11, tm_mday=8, tm_hour=13, tm_min=20, tm_sec=12, tm_wday=5, tm_yday=312, tm_isdst=0)